Search Engine tool with Langchain Agent

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [3]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tool=[wiki, arxiv]

In [6]:
# custom tool 
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

e:\GenAI\Search Engine with Langchain Tools and Agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, embeddings)
retriever=vectordb.as_retriever()
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D90CF6E120> search_kwargs={}


In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [10]:
tool=[wiki, arxiv, retriever_tool]

In [12]:
# Run this tool with agent and LLM models
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
hf_api_key=os.getenv("HF_API_KEY")

from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")


In [15]:
# prompt template 
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

e:\GenAI\Search Engine with Langchain Tools and Agent\venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
# Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tool, prompt)
print(agent)

first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
}) middle=[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_c

In [18]:
# agent executor

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent, tools=tool, verbose=True)
print(agent_executor)

verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMess

In [19]:
agent_executor.invoke({"input":"Tell me something about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If 

{'input': 'Tell me something about Langsmith',
 'output': 'Based on the result of the tool call, Langsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. Langsmith integrates seamlessly with other platforms such as LangChain and LangGraph, and can be used standalone as well.\n\nSome of the key features of Langsmith include:\n\n* Logging traces and evaluating application performance\n* Monitoring and evaluating applications\n* Integrating with other platforms such as LangChain and LangGraph\n* Providing multiple ways to log traces, including the use of `traceable()`\n* Offering tutorials, conceptual guides, and how-to guides to help users get started\n\nOverall, Langsmith appears to be a powerful tool for building and deploying LLM applications, and provides a range of features to help users monitor and evaluate their applications.'

In [20]:
agent_executor.invoke({"input":"How is Machine Learning shaping our future"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'How is Machine Learning shaping our future'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech re
Invoking: `langsmith-search` with `{'query': 'Machine learning shaping our future'}`


Trace LLM Applications: Gain visibility into LLM calls and other parts of your application's logic.
Evaluate Performance: Compare results across models, prompts, and architectures to identify what works best.
Improve Prompts: Quickly refine prompts to achieve more accurate and reliab

{'input': 'How is Machine Learning shaping our future',
 'output': 'It seems like the tool call yielded a result that is not directly related to the question about Machine Learning shaping our future. The result appears to be a documentation page for a platform called LangSmith, which is focused on building and evaluating Large Language Models (LLMs).\n\nSince the tool call did not yield a relevant result, I will respond directly to the question.\n\nMachine Learning (ML) is shaping our future in numerous ways. Its applications are diverse and widespread, and it has the potential to revolutionize various industries. Some of the ways ML is shaping our future include:\n\n1. Automation: ML is being used to automate tasks, such as data processing, image recognition, and natural language processing, which can improve efficiency and reduce errors.\n2. Healthcare: ML is being used in healthcare to develop personalized medicine, diagnose diseases more accurately, and develop new treatments.\n3.

In [21]:
agent_executor.invoke({"input":"Tell me something about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech re
Invoking: `wikipedia` with `{'query': 'Machine learning#Applications'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Advances in the field of deep learning have a

{'input': 'Tell me something about Machine Learning',
 'output': "I don't think we need to use any more tools for this conversation. The information provided about LangSmith seems to be sufficient. Thank you for the information!"}

In [22]:
agent_executor.invoke({"input":"What is this research paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, baseBased on the output from the tool call, it appears that the research paper 1706.03762 is titled "Attention Is All You Need" and was published on August 2, 2023. The paper was written by a team of authors and discusses a new simple network architecture called the Transformer, which is used for sequence transduction models.

> Finished chain.


{'input': 'What is this research paper 1706.03762 about?',
 'output': 'Based on the output from the tool call, it appears that the research paper 1706.03762 is titled "Attention Is All You Need" and was published on August 2, 2023. The paper was written by a team of authors and discusses a new simple network architecture called the Transformer, which is used for sequence transduction models.'}